In [1]:

from math import log

from Phase_1.src.Utils.SimplePositionalIndex import SimplePositionalIndex
from Phase_1.src.Utils.utilities import read_file

docs_url, docs_title, docs_content = read_file()


class DevelopedPositionalIndex(SimplePositionalIndex):
    def __init__(self, documents_url, documents_title, documents_content):
        super().__init__(documents_url, documents_title, documents_content, False)
        self.total_number_of_documents = len(documents_url)
        self.build_updated_positional_index()

    def build_updated_positional_index(self):
        for WORD in self.positional_index_structure.keys():
            number_of_unique_occurrences = len(self.positional_index_structure[WORD]['indexes'])
            for TERM_URL, DICTIONARY in self.positional_index_structure[WORD]['indexes'].items():
                DICTIONARY['tf idf'] = self.get_tf_value(WORD, TERM_URL) * self.get_idf_value(WORD)

    def get_tf_value(self, word, url):
        return 1 + log(self.positional_index_structure[word]['indexes'][url]['number of occurrences in document'])

    def get_idf_value(self, word):
        return log(self.total_number_of_documents / len(self.positional_index_structure[word]['indexes']))


pos_index = DevelopedPositionalIndex(documents_url=docs_url, documents_title=docs_title, documents_content=docs_content)


In [2]:
for word in pos_index.positional_index_structure.keys():
    for doc_url, dictionary in pos_index.positional_index_structure[word]['indexes'].items():
        print(dictionary['tf idf'])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



8.310742863043687
8.310742863043687
8.310742863043687
9.409355151711798
6.924448501923798
6.924448501923798
6.924448501923798
6.924448501923798
6.924448501923798
6.924448501923798
6.924448501923798
6.924448501923798
6.924448501923798
6.924448501923798
6.924448501923798
6.924448501923798
8.310742863043687
8.310742863043687
8.310742863043687
8.310742863043687
8.310742863043687
8.310742863043687
8.310742863043687
8.310742863043687
8.310742863043687
9.409355151711798
9.409355151711798
9.409355151711798
9.409355151711798
8.716207971151853
8.716207971151853
8.716207971151853
8.716207971151853
9.409355151711798
8.310742863043687
8.310742863043687
8.310742863043687
9.409355151711798
9.409355151711798
9.409355151711798
9.409355151711798
9.409355151711798
6.844405794250261
6.844405794250261
6.844405794250261
6.844405794250261
6.844405794250261
14.363754108444814
6.844405794250261
6.844405794250261
6.844405794250261
6.844405794250261
11.588586373142984
11.588586373142984
6.844405794250261
5.79843

In [13]:
from collections import Counter
from Phase_1.src.Utils.utilities import preprocess_pipeline



class QueryHandler:
    def __init__(self, positional_index):
        self.positional_index = positional_index

    def answer_query(self, query):
        query = preprocess_pipeline(query)
        terms = query.split()
        print(self.positional_index.total_number_of_documents)
        vector_values = self.tf_idf_calculate(terms)

    def tf_idf_calculate(self, terms):
        vector_values = {}
        tf_values = Counter(terms)
        for term in terms:
            positional_index_structure = self.positional_index.positional_index_structure
            vector_values[term] = {}
            if term in positional_index_structure.keys():
                for DOC_URL in positional_index_structure[term]['indexes'].keys():
                    vector_values[term][DOC_URL] = \
                        (1 + log(tf_values[term])) * self.positional_index.get_idf_value(term)
        return vector_values

    def cosine_similarity(self, vector_values):
        pass


In [14]:
query_handler = QueryHandler(pos_index)

query_handler.answer_query('درود بر کیر خر')

12202
